## Train and Deploy CNN Models

##Download the data

In [ ]:
#Download the dataset from https://www.kaggle.com/c/diabetic-retinopathy-detection/data

##Import the libraries

##Define the training strategy

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
with strategy.scope():
    diabetic_model = createModel()
    diabetic_model.fit_generator(aug.flow(Xtrain,
                                          trainY, batch_size=BS),
                                          validation_data=(Xval, valY),
                                          steps_per_epoch=len(trainX) // BS,
                                          epochs=EPOCHS,
                                          verbose=1)

In [ ]:
# print the summary of model
from keras.utils import print_summary
print_summary(diabetic_model, line_length=None, positions=None, print_fn=None)

##Save the model to disk

In [ ]:
import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
CWD = os.getcwd()
MODEL_DIR = os.path.join(CWD, 'diabetic_model')
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

In [ ]:
tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None
)

##Serve the model with TensorFlow Serving



In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
!docker pull tensorflow/serving

'docker' is not recognized as an internal or external command,
operable program or batch file.


#### Run the docker container

In [ ]:
!docker run -p 8501:8501 --name tfserving_resnet \
--mount type=bind,source=${MODEL_DIR},target=/models/diabetic_model \
-e MODEL_NAME=diabetic_model -t tensorflow/serving &